In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm, trange

from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error
import math

from sklearn.metrics.pairwise import cosine_similarity

from scipy import sparse

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [ ]:
#lấy thông tin user
user_cols = ['id','age','sex','job','zip_code']
users = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/u.user',sep = '|', names = user_cols, encoding = 'latin-1')
users

,id,age,sex,job,zip_code
0,1,24,M,technician,85711
1,2,53,F,other,94043
2,3,23,M,writer,32067
3,4,24,M,technician,43537
4,5,33,F,other,15213
...,...,...,...,...,...
938,939,26,F,student,33319
939,940,32,M,administrator,02215
940,941,20,M,student,97229
941,942,48,F,librarian,78209


In [ ]:
#Lấy tt item
i_cols = ['movie id', 'movie title' ,'release date','video release date', 'IMDb URL', 'unknown', 'Action', 'Adventure','Animation', 'Children\'s', 'Comedy', 'Crime', 'Documentary', 'Drama', 'Fantasy','Film-Noir', 'Horror', 'Musical', 'Mystery', 'Romance', 'Sci-Fi', 'Thriller', 'War', 'Western']
items = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/u.item',sep = '|',names = i_cols, encoding = 'latin-1' )
items

,movie id,movie title,release date,video release date,IMDb URL,unknown,Action,Adventure,Animation,Children's,Comedy,Crime,Documentary,Drama,Fantasy,Film-Noir,Horror,Musical,Mystery,Romance,Sci-Fi,Thriller,War,Western
0,1,Toy Story (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Toy%20Story%2...,0,0,0,1,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0
1,2,GoldenEye (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?GoldenEye%20(...,0,1,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,3,Four Rooms (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Four%20Rooms%...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
3,4,Get Shorty (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Get%20Shorty%...,0,1,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0
4,5,Copycat (1995),01-Jan-1995,NaN,http://us.imdb.com/M/title-exact?Copycat%20(1995),0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,Mat' i syn (1997),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?Mat%27+i+syn+...,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0
1678,1679,B. Monkey (1998),06-Feb-1998,NaN,http://us.imdb.com/M/title-exact?B%2E+Monkey+(...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0
1679,1680,Sliding Doors (1998),01-Jan-1998,NaN,http://us.imdb.com/Title?Sliding+Doors+(1998),0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0
1680,1681,You So Crazy (1994),01-Jan-1994,NaN,http://us.imdb.com/M/title-exact?You%20So%20Cr...,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0


In [ ]:
#Tập matrix rating
r_cols = ['user_id', 'movie_id', 'rating', 'unix_timestamp']

ratings_base = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/ua.base', sep='\t', names=r_cols, encoding='latin-1')
ratings_test = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/ua.test', sep='\t', names=r_cols, encoding='latin-1')

In [ ]:
ratings_test

,user_id,movie_id,rating,unix_timestamp
0,1,20,4,887431883
1,1,33,4,878542699
2,1,61,4,878542420
3,1,117,3,874965739
4,1,155,2,878542201
...,...,...,...,...
9425,943,232,4,888639867
9426,943,356,4,888639598
9427,943,570,1,888640125
9428,943,808,4,888639868


#Content Based

Thử nghiệm, đánh giá phương pháp và áp dụng cho next item


In [ ]:
#feature vector chỉ xem xét feature, ở đây là tag của các movie, nên lấy các giá trị nhị phân ở cuối
X = np.array(items)[:,-19:]

In [ ]:
#Lấy các item mà user đã rated và giá trị rate của user ở điểm đó.
def check_rated_item(rating,user_id):
  item_ids = np.array([rating['movie_id'][x] for x in range(len(rating)) if rating['user_id'][x] == user_id ])# lọc dựa trên tìm kiếm cột user_id có giá trị bằng với user id truyền vào trong module
  ratings = np.array([rating['rating'][x] for x in range(len(rating)) if rating['user_id'][x] == user_id ]).reshape(-1,1)

  return item_ids, ratings

Áp dụng ridge regression cho từng user.

In [ ]:
#Tạo hàm tạo model kết quả trả về là w và b
def model_ridge(X,Y):
  clf = Ridge(alpha=0.01, fit_intercept  = True)
  clf.fit(X, Y) 

  return clf.coef_, clf.intercept_

In [ ]:
#áp dụng mô hình cho từng user và lưu w,b của từng user vào users
#Sau khi chạy model xong, các trọng số đã được save lại vào một file csv
#để mỗi lần chạy khác ko cần tốn thời gian train lại

W = []
b = []
for x in tqdm(range(len(users))):
  item_ids, ratings = check_rated_item(ratings_base,users['id'][x])
  coef_,intercept_ = model_ridge(X[item_ids-1],ratings)
  W.append(coef_)
  b.append(intercept_)

users['coef_'] = [np.array(x) for x in W]
users['intercept_'] = [np.array(x) for x in b]

users

,Unnamed: 0,id,age,sex,job,zip_code,coef_,intercept_
0,0,1,24,M,technician,85711,"[[0.5331712271191822, -0.3150830106977706, -0....",[3.461497060609626]
1,1,2,53,F,other,94043,"[[0.0, 1.3950819215722088, -0.0550670775127266...",[2.1306923109839113]
2,2,3,23,M,writer,32067,"[[0.0, 0.2968658402371606, 1.4141607110000722,...",[1.8953353069223726]
3,3,4,24,M,technician,43537,"[[0.0, -0.0240081928061864, -1.979198868941134...",[5.026768570786242]
4,4,5,33,F,other,15213,"[[1.642816821284645, -0.001764301221320541, 0....",[2.3407550105025017]
...,...,...,...,...,...,...,...,...
938,938,939,26,F,student,33319,"[[0.0, 0.09241365009587692, -1.0000636540765, ...",[3.5300748622340814]
939,939,940,32,M,administrator,02215,"[[0.0, 0.7943916396396595, -0.8976695460964073...",[3.2017072900877492]
940,940,941,20,M,student,97229,"[[0.0, -2.900279476274026, -2.9098810745379105...",[6.877972083552748]
941,941,942,48,F,librarian,78209,"[[0.0, -0.19601970375010258, 0.729758345183620...",[3.9500368485853707]


In [ ]:
#Save point 
# users.to_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/user.csv')
users = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/user.csv')

#do khi save vào file csv, cột coef_ và intercept_ bị chuyển thành định dạng string 
#Xử lý lại thành dạng array

#đoạn code chỉ chạy một lần
# for i in range(len(users)):
#   coef_ = users.coef_[i]
#   for x in ['[[',']]','\n']:
#     coef_ = coef_.replace(x,'')
#   coef_ = coef_.split()
#   coef_ = [float(x) for x in coef_]
#   users.coef_[i] = np.array(coef_)

#   intercept_ = users.intercept_[i]
#   intercept_ = intercept_[1:-1]
#   intercept_ = float(intercept_)
#   users.intercept_[i] = np.array(intercept_)

,Unnamed: 0,Unnamed: 0.1,id,age,sex,job,zip_code,coef_,intercept_
0,0,0,1,24,M,technician,85711,[[ 0.53317123 -0.31508301 -0.4818312 0.55453...,[3.46149706]
1,1,1,2,53,F,other,94043,[[ 0. 1.39508192 -0.05506708 -1.29169...,[2.13069231]
2,2,2,3,23,M,writer,32067,[[ 0. 0.29686584 1.41416071 0. ...,[1.89533531]
3,3,3,4,24,M,technician,43537,[[ 0. -0.02400819 -1.97919887 0. ...,[5.02676857]
4,4,4,5,33,F,other,15213,[[ 1.64281682e+00 -1.76430122e-03 4.73704779e...,[2.34075501]
...,...,...,...,...,...,...,...,...,...
938,938,938,939,26,F,student,33319,[[ 0. 0.09241365 -1.00006365 0. ...,[3.53007486]
939,939,939,940,32,M,administrator,02215,[[ 0. 0.79439164 -0.89766955 1.26586...,[3.20170729]
940,940,940,941,20,M,student,97229,[[ 0. -2.90027948 -2.90988107 1.29313...,[6.87797208]
941,941,941,942,48,F,librarian,78209,[[ 0. -0.1960197 0.72975835 0.86211...,[3.95003685]


In [ ]:
# đánh giá mô hình
pred = []
true = []
for x in range(len(ratings_test)):
  user_id = ratings_test['user_id'][x] - 1 #do việc đánh số ở Users cũng bị lêch xuống 1
  movie_id = ratings_test['movie_id'][x] - 1 #do việc đánh số ở X bị lệch xuống 1
  y = X[ movie_id ].dot(users['coef_'][user_id].reshape(19,)) + users['intercept_'][user_id]

  pred.append(y)
  true.append(ratings_test['rating'][x])

RMSE = math.sqrt(mean_squared_error(true,pred))
print(RMSE)

1.2760653674169282


In [ ]:
# #Ranking các bộ phim theo user.

def ranking(user_id, users, data_movie, data_rating  ):
  movie_id, rating = check_rated_item(data_rating, user_id)#lấy được movie mà user đã rate

  rank = {}
  #ranking
  for i in movie_id:
    y = data_movie[ i - 1 ].dot(users['coef_'][user_id -1].reshape(19,)) + users['intercept_'][user_id-1]
    rank[i] = y
  rank = sorted(rank.items(), key=lambda x: x[1], reverse=True)
  return rank

In [ ]:
#dự đoán item cho user
user_id = 100
rank = ranking(user_id, users, X, ratings_test)
rank

[(302, 4.423234450000001),
 (268, 3.3063925100000002),
 (354, 3.0551638700000003),
 (355, 2.85786882),
 (340, 2.8468141300000003),
 (344, 2.8468141300000003),
 (750, 2.8468141300000003),
 (321, 2.5955854900000004),
 (266, 2.33221969),
 (288, 2.3100178600000003)]

#Collaborative Filtering

## User-user 

In [ ]:
# Xây dựng utility Maxtrix 
# movie là hàng, user cột

def utility_matrix(users, items, rating):
  #tao df toan khong
  col = [x for x in users['id']]
  col.insert(0,'movie_id')
  a = np.zeros(shape=(len(items),len(users) + 1))
  df = pd.DataFrame(a,columns = col)

  #thay them user id vào cột user_id
  df['movie_id'] = items['movie id']

  # #xay dung ulity matrix
  for x in tqdm(range(len(rating))):
    user_id = rating.user_id[x]  #do bị lệch 1
    movie_id = rating.movie_id[x] - 1
    rate = rating.rating[x]

    df[user_id][movie_id] = rate

  return df

# chuẩn hóa utility matrix
def normalize_utility(df_1):
  df = df_1.copy()
  df = df.replace(0, np.NaN)

  mean_arr = np.array(df.iloc[:,1:].mean(axis = 0))#mảng mà ở đó vị trí i tưởng ứng với user i+1
  for i in range(len(mean_arr)):
    df[i+1] = df[i+1] - mean_arr[i]
  df = df.replace(np.NaN,0)

  return df


In [ ]:
df = utility_matrix(users, items, ratings_base)
df = normalize_utility(df)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
#save point
# df.to_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/utility.csv')

df = pd.read_csv('/content/gdrive/MyDrive/Đại học/Data/ml-100k/utility.csv')
df = df.iloc[:,1:]
df

,movie_id,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,...,904,905,906,907,908,909,910,911,912,913,914,915,916,917,918,919,920,921,922,923,924,925,926,927,928,929,930,931,932,933,934,935,936,937,938,939,940,941,942,943
0,1,1.396947,0.25,0.0,0.0,0.0,0.368159,0.000000,0.0,0.0,-0.201149,0.0,0.000000,-0.089457,0.0,-1.957447,0.669231,0.0,1.11985,0.0,-0.210526,2.343195,0.000000,1.361702,0.0,0.985294,0.061856,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.252252,0.0,...,0.0,0.0,0.0,0.394161,0.0,0.0,0.94,0.0,0.0,-1.54918,0.0,0.0,0.625407,-0.4,-0.333333,0.531401,0.0,0.0,1.632479,-1.171875,1.236111,0.0,0.0,1.345455,0.0,-0.641026,-0.056604,0.0,0.038961,0.408046,-1.707317,-0.965517,0.257576,0.0,0.693878,0.0,0.000000,1.166667,0.0,0.000000
1,2,-0.603053,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,-0.089457,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,-1.338983,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,-0.787879,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.0,-0.374593,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,-0.763889,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.292683,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,1.607595
2,3,0.396947,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,-1.06,0.0,0.0,0.00000,0.0,0.0,-0.374593,-2.4,0.000000,0.000000,0.0,0.0,0.000000,-0.171875,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.257576,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
3,4,-0.603053,0.00,0.0,0.0,0.0,0.000000,1.040712,0.0,0.0,-0.201149,0.0,0.658537,1.910543,0.0,0.000000,0.669231,0.0,-0.88015,0.0,0.000000,0.000000,1.661017,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.45082,0.0,0.0,0.625407,0.0,0.000000,-2.468599,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.408046,1.292683,0.000000,0.000000,0.0,0.000000,0.0,-1.463918,0.000000,0.0,0.000000
4,5,-0.603053,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,-2.089457,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,-0.656805,0.000000,0.000000,0.0,0.000000,0.000000,0.0,-0.710145,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.394161,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.0,-0.374593,0.0,0.000000,0.531401,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1677,1678,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.0,0.00000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000
1678,1679,0.000000,0.00,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.00000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.0,0.0,

In [ ]:
#hàm tính similarity giữa 2 user

def similarity_func(user_id, df, K):
  #loai bo cot movie_id do các gia tri cung hang sẽ thuộc cùng 1 bộ phim nên việc loại bỏ ko  ảnh hưởng
  a = np.array(df.iloc[:,1:])
  neighbor_vectors = np.transpose(a) #chuyen sang dạng vector user(hàng)-movie(cột) để dùng cosine_simi

  #lấy user vector
  user_vector = np.array([neighbor_vectors[user_id-1]])
   
  #tinh similarity cosine 
  similarity = cosine_similarity(neighbor_vectors,user_vector)

  #tao ma tran danh so
  user_ids = np.arange(1,len(neighbor_vectors)+1).reshape(-1,1)

  #tao matran ranking similarity [userid,similarity]
  simi = np.concatenate((user_ids, similarity), axis=1)

  simi_ranked = simi.copy()
  simi_ranked = simi_ranked[simi_ranked[:, 1].argsort()]#sắp xếp theo chiều tang của similarity
  simi_ranked = simi_ranked[::-1]

  
  #ranking san pham [id_item, rating]
  #for trong tung bo phim, check rate của phim nao con thieu
  # for tung user, check user nao da rate phim i roi, đưa vào trong 1 mang
  #loai bo nhung mang nao ko du so ptu de danh gia
  # tính rating từ các ptu trong mang do.

  ranking_movie = []

  for i in tqdm(range(len(user_vector[0]))):
    if user_vector[0,i] == 0:
      k = 0
      nearest_neighbor = []

      for j in range(1, len(simi)): #do vi tri 0 la chinh user do
        neigbor_id = int(simi_ranked[j,0] - 1)
        if neighbor_vectors[neigbor_id,i] != 0:
          nearest_neighbor.append(neigbor_id)
          k+=1
        if k == K:
          break


      if k == K:
        nearest_neighbor = np.array(nearest_neighbor)    
        sum_para = 0
        sum_sim = 0
        for j in range(len(nearest_neighbor)):
            
          sum_para += neighbor_vectors[nearest_neighbor[j], i]*simi[nearest_neighbor[j],1]
          sum_sim += simi[nearest_neighbor[j],1]
        rate = sum_para/sum_sim

        ranking_movie.append([i+1, rate])

  ranking_movie = np.array(ranking_movie)
  ranking_movie = ranking_movie [ranking_movie [:, 1].argsort()]
  
  return ranking_movie[::-1]
   

In [ ]:
user_id = 10
k = 10 # k là số user gần kề được chọn để đánh giá
similarity_func(user_id, df, k)



[[ 1.00000000e+01  1.00000000e+00]
 [ 3.21000000e+02  2.12775135e-01]
 [ 7.10000000e+02  2.07612661e-01]
 ...
 [ 4.31000000e+02 -1.01744184e-01]
 [ 7.53000000e+02 -1.28145510e-01]
 [ 7.77000000e+02 -1.41704905e-01]]


array([[ 6.44000000e+02,  1.09143520e+00],
       [ 1.14300000e+03,  1.04418087e+00],
       [ 3.20000000e+02,  1.03769729e+00],
       ...,
       [ 4.57000000e+02, -2.18022576e+00],
       [ 6.88000000e+02, -2.40460289e+00],
       [ 1.08700000e+03, -2.68871451e+00]])